AlexNet (2012)

배경  
기계 학습의 성능을 높이기 위해서는 더 큰 데이터 셋이 필요하다.  
이 큰 데이터를 잘 학습하기 위해서는 용량이 큰 모델이 필요한데, CNN을 활용했다.  
CNN은 비슷한 규모의 피드포워드 신경망보다 간선과 파라미터가 적으므로 학습이 쉽다.  
물론 best-perfomance는 조금 떨어질 수 있다.

그럼에도 불구하고 GPU용량이 부족해서 두대의 GPU에 커널(파라미터)을 분산시켜서 학습했다.  

이 모델은 일정한 입력이 필요하므로 데이터를 256x256으로 다운 샘플링 해서 사용하였다.

최종 모델은 5개의 Conv층과 3개의 FC층으로 이루어져 있고, 여기서 Conv층을 조금이라도 제외한다면 성능이 떨어짐을 확인했다.  

- ReLU를 사용: tanh와 같은 포화 비선형 함수보다 비포화 비선형 함수인 ReLU를 사용했을 때 훨씬 빠른 학습이 가능했다. 다른 기존 모델들은 과적합 방지에 초점을 두었지만, 이 논문에서는 ReLU를 도입하므로써 학습의 가속에 초점을 두었다.
- 두개의 GPU를 사용: 파라미터를 두개의 GPU에서 따로 계산하지만 특정 층에서 이를 공유하여 교차 학습하였다. 이는 하나의 GPU에서 훈련된 것보다 오류가 낮았다.
- LRN: ReLU는 활성화 된 값이 포화되지 않으므로 하나의 파라미터가 다른 파라미터에 영향을 끼칠 수 있음. 입력 정규화 대신 지역 정규화를 사용하여 위 문제로 발생하는 일반화를 해결.
- Overlapping Pooling: Stride를 2, 커널 사이즈는 3으로 하여 커널이 조금씩 겹치게함 -> 과적합 방지에 도움이 되었다.

과적합 방지
- 데이터 확대 1: 사진 하나로 5개의 224x224패치를 만들고 수평 반사해서 10장으로 만듦. 10개의 부분에 대한 예측을 평균해서 예측 출력
- 데이터 확대 2: 객체의 속성은 조명이나 색상의 영향이 적다고 판단해 RGB값을 PCA를 수행하여 이미지 생성
- 드롭아웃: 첫 두개의 FC에서 드롭아웃 적용


In [ ]:
# code with Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(11),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(5),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256*6*6, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 6)
        )
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x,1)
        x = self.classifier(x)
        return x